In [1]:
#Importing libraries
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow import keras


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:


import os
import random
import shutil

# Set the paths for the main directory, train directory, and test directory
main_directory = '/content/drive/MyDrive/Projects/Face covering classification/glasses-and-coverings'
train_set = '/content/drive/MyDrive/Projects/Face covering classification/train'
test_set = '/content/drive/MyDrive/Projects/Face covering classification/test'

# Create the train and test directories if they don't exist
os.makedirs(train_set, exist_ok=True)
os.makedirs(test_set, exist_ok=True)

# Set the split ratio for train and test (80% train, 20% test)
split_ratio = 0.8

# Iterate through each folder in the main directory
for folder_name in os.listdir(main_directory):
    folder_path = os.path.join(main_directory, folder_name)
    if not os.path.isdir(folder_path):
        continue

    # Create the corresponding subdirectories in the train and test directories
    train_folder_path = os.path.join(train_set, folder_name)
    test_folder_path = os.path.join(test_set, folder_name)
    os.makedirs(train_folder_path, exist_ok=True)
    os.makedirs(test_folder_path, exist_ok=True)

    # List all the image files in the current folder
    image_files = [
        file_name
        for file_name in os.listdir(folder_path)
        if os.path.isfile(os.path.join(folder_path, file_name))
           and file_name.lower().endswith((".jpg", ".jpeg", ".png"))
    ]

    # Shuffle the list of image files randomly
    random.shuffle(image_files)

    # Determine the number of images for the train and test sets
    split_index = int(len(image_files) * split_ratio)

    # Move the train set files to the train folder
    for file_name in image_files[:split_index]:
        src_path = os.path.join(folder_path, file_name)
        dest_path = os.path.join(train_folder_path, file_name)
        shutil.move(src_path, dest_path)

    # Move the test set files to the test folder
    for file_name in image_files[split_index:]:
        src_path = os.path.join(folder_path, file_name)
        dest_path = os.path.join(test_folder_path, file_name)
        shutil.move(src_path, dest_path)

In [5]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Projects/Face covering classification/train',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 1566 images belonging to 4 classes.


In [6]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Projects/Face covering classification/test',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 395 images belonging to 4 classes.


In [7]:
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [8]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25


/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py:1176: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


49/49 [==============================] - 872s 18s/step - loss: 0.0000e+00 - accuracy: 0.2682 - val_loss: 0.0000e+00 - val_accuracy: 0.2684
Epoch 2/25
49/49 [==============================] - 13s 274ms/step - loss: 0.0000e+00 - accuracy: 0.2682 - val_loss: 0.0000e+00 - val_accuracy: 0.2684
Epoch 3/25
49/49 [==============================] - 14s 292ms/step - loss: 0.0000e+00 - accuracy: 0.2682 - val_loss: 0.0000e+00 - val_accuracy: 0.2684
Epoch 4/25
49/49 [==============================] - 14s 277ms/step - loss: 0.0000e+00 - accuracy: 0.2682 - val_loss: 0.0000e+00 - val_accuracy: 0.2684
Epoch 5/25
49/49 [==============================] - 13s 269ms/step - loss: 0.0000e+00 - accuracy: 0.2682 - val_loss: 0.0000e+00 - val_accuracy: 0.2684
Epoch 6/25
49/49 [==============================] - 13s 273ms/step - loss: 0.0000e+00 - accuracy: 0.2682 - val_loss: 0.0000e+00 - val_accuracy: 0.2684
Epoch 7/25
49/49 [==============================] - 14s 278ms/step - loss: 0.0000e+00 - accuracy: 0.2682 -

In [10]:
import numpy as np
from keras.preprocessing import image
test_image = tf.keras.utils.load_img('/content/drive/MyDrive/Projects/Face covering classification/test/plain/2017-11-30-18-15-41.jpg', target_size = (64, 64))
test_image = tf.keras.utils.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 0:
  prediction = 'covering'
elif result[0][0] == 1:
  prediction = 'glasses'
elif result[0][0] == 2:
  prediction = 'plain'
elif result[0][0] == 3:
  prediction = 'sunglasses'

print(prediction)

1/1 [==============================] - 0s 21ms/step
glasses
